In [1]:
using Distributed
addprocs(4)

4-element Vector{Int64}:
 2
 3
 4
 5

In [2]:
@everywhere struct Parm
    #Parm of electrons
    t_i::Float64
    t_e::Float64
    t_aa::Float64
    t_bb::Float64
    Delta::Float64
    mu::Float64
    eta::Float64
    hx::Float64
    hy::Float64
    hz::Float64

    #Parm of phonon
    Cp::Float64
    m::Float64
    M::Float64
    U::Float64
    T::Float64
    K_SIZE::Int
    W_MAX::Float64
    W_SIZE::Int
    W_in::Float64
end

In [3]:
#@everywhere 
using SharedArrays

In [4]:
#@everywhere 
mutable struct Hamiltonian
    Hk::SharedArray{ComplexF64,3}
    Vx::SharedArray{ComplexF64,3}
    Vxx::SharedArray{ComplexF64,3}
    E::SharedArray{ComplexF64,2}
    #=
    Hk::SharedArray{Array{ComplexF64,2}}
    Vx::SharedArray{Array{ComplexF64,2}}
    Vxx::SharedArray{Array{ComplexF64,2}}
    E::SharedArray{Array{ComplexF64,2}}=#
end

In [5]:
@everywhere sigma = [[1.0 0.0; 0.0 1.0], [0.0 1.0; 1.0 0.0], [0.0 -1.0im; 1.0im 0.0], [1.0 0.0; 0.0 -1.0]]

In [48]:
H0 = Array{ComplexF64,3}(undef, 10, 2, 2)
#Array{Array{ComplexF64,1},1}(undef, 10)
#Array{Matrix{ComplexF64},1}

10×2×2 Array{ComplexF64, 3}:
[:, :, 1] =
 9.12119e-316+9.12119e-316im  9.12119e-316+9.12119e-316im
 9.12119e-316+9.12119e-316im  9.12119e-316+1.42298e-315im
 9.12119e-316+9.12119e-316im  9.12119e-316+9.12119e-316im
 9.12119e-316+9.12119e-316im  9.12119e-316+9.12119e-316im
 9.12119e-316+9.12119e-316im  1.42298e-315+9.12119e-316im
 9.12119e-316+9.12119e-316im  9.12119e-316+9.12119e-316im
 9.12119e-316+9.12119e-316im  9.12119e-316+1.42298e-315im
 9.12119e-316+9.12119e-316im  9.12119e-316+9.12119e-316im
 9.12119e-316+9.12119e-316im  1.42298e-315+9.12119e-316im
 9.12119e-316+9.12119e-316im  9.12119e-316+9.12119e-316im

[:, :, 2] =
 1.04249e-315+9.12119e-316im  9.12119e-316+9.12119e-316im
 9.12119e-316+9.12119e-316im  9.12119e-316+9.12119e-316im
 9.12119e-316+9.12119e-316im  9.12119e-316+9.12119e-316im
 9.12119e-316+9.12119e-316im  9.12119e-316+9.12119e-316im
 9.12119e-316+9.12119e-316im  9.12119e-316+9.12119e-316im
 9.12119e-316+9.12119e-316im  1.42298e-315+9.12119e-316im
 9.12119e-316+9.12

In [50]:
H0[1,:,:] = [1.0 2.0; 1.0 1.0]

2×2 Matrix{Float64}:
 1.0  2.0
 1.0  1.0

In [ ]:
function test(p::Parm)
    a::SharedArray
end

In [16]:
#@everywhere 
function HandV(p::Parm)
    Hk = SharedArray{ComplexF64,3}(p.K_SIZE,2,2)
    Vx = SharedArray{ComplexF64,3}(p.K_SIZE,2,2)
    Vxx = SharedArray{ComplexF64,3}(p.K_SIZE,2,2)
    E = SharedArray{ComplexF64,2}(p.K_SIZE,2)

    @distributed for k in 1:p.K_SIZE
        k0::Float64 = 2.0*(k - p.K_SIZE/2)*pi/p.K_SIZE
        
        eps::Float64 = -0.5*(p.t_aa + p.t_bb)*cos(k0)-p.mu
        g_x::Float64 = (p.t_e*cos(k0) -p.t_i*sin(k0)) - p.hx
        g_y::Float64 = (p.t_e*sin(k0) + p.t_i*cos(k0)) - p.hy
        g_z::Float64 = p.Delta - p.hz -0.5*(p.t_aa - p.t_bb)*cos(k0)
        gg = [eps, g_x, g_y, g_z] 
        Hk[k,:,:] = gg' * sigma

        eps_vx::Float64 = 0.5*(p.t_aa + p.t_bb)*sin(k0)
        gx_vx::Float64 = (-p.t_e*sin(k0) -p.t_i*cos(k0))
        gy_vx::Float64 = (p.t_e*cos(k0) - p.t_i*sin(k0))
        gz_vx::Float64 = 0.5*(p.t_aa - p.t_bb)*sin(k0)
        gg_x = [eps_vx, gx_vx, gy_vx, gz_vx]
        Vx[k,:,:] = gg_x' * sigma

        eps_vxx::Float64 = 0.5*(p.t_aa + p.t_bb)*cos(k0)
        gx_vxx::Float64 = (-p.t_e*cos(k0) +p.t_i*sin(k0))
        gy_vxx::Float64 = (-p.t_e*sin(k0) - p.t_i*cos(k0))
        gz_vxx::Float64 = 0.5*(p.t_aa - p.t_bb)*cos(k0)
        gg_xx = [eps_vxx, gx_vxx, gy_vxx, gz_vxx]
        Vxx[k,:,:] = gg_xx' * sigma

        E[k,:] = zeros(2)
    end

    return Hk, Vx, Vxx, E
end

HandV (generic function with 1 method)

In [7]:
#Parm(t_i, t_e, t_aa, t_bb, Delta, mu, eta, hx, hy, hz, Cp, m, M, U, T, K_SIZE, W_MAX, W_SIZE, W_in)
@everywhere p = Parm(1.0, 0.3, 0.1, 0.2, 0.1, -0.2, 0.02, 0.6, 0.0, 0.0, 0.5, 0.5, 1.0, 0.2, 0.02, 20, 1.5, 20, 1.0)

In [17]:
#Hk, Vx, Vxx, E = HandV(p)
H = Hamiltonian(HandV(p)...)

Hamiltonian([0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im;;; 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im], [0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im;;; 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im], [0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im;;; 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im], ComplexF64[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im])

In [18]:
H.Hk[11,:,:]

2×2 Matrix{ComplexF64}:
 0.204894+0.0im         -0.6237-1.04376im
  -0.6237+1.04376im  -0.0902113+0.0im